In [1]:
from sogaPreprocessor import *
from producecfg import *
from libSOGA import *

In [284]:
compiledFile=compile2SOGA('../programs/SOGA/BernoulliTorch.soga')
cfg = produce_cfg(compiledFile)
output_dist = start_SOGA(cfg)

NameError: name 'np' is not defined

In [ ]:
output_dist.gm.marg_pdf(torch.tensor(0.001), 1)

### Example of gradient-based optimization

In [2]:
compiledFile=compile2SOGA('../programs/SOGA/Optim.soga')
cfg = produce_cfg(compiledFile)

In [3]:
pars = {'par':1.}

# creates torch parameters for each params declared in the dictionary
params_dict = {}
for key, value in pars.items():
    params_dict[key] = torch.tensor(value, requires_grad=True)    

In [4]:
output_dist = start_SOGA(cfg)

None
0


In [280]:
samples = distributions.Normal(torch.tensor(10.), torch.tensor(2.)).sample((100,1))

In [281]:
def neg_likelihood(dist, samples):
    likelihood = torch.tensor([0.])
    for sample in samples:
        likelihood += dist.gm.marg_pdf(sample,0)
    return -likelihood

In [282]:
import torch

# Define the optimizer with init_mean as the parameter

optimizer = torch.optim.SGD([params_dict[key] for key in params_dict.keys()], lr=0.1)

for i in range(100):
    optimizer.zero_grad()  # Reset gradients
    
    # loss
    #current_dist = compute_posterior(init_mean)
    current_dist = start_SOGA(cfg, params_dict)
    loss = neg_likelihood(current_dist, samples)
    
    # Backpropagate
    loss.backward()
    
    # Update init_mean
    optimizer.step()

    #with torch.no_grad():
    #    params_dict['std'].clamp_(0.001, 1e10)
    
    # Print progress
    if i % 10 == 0:
        out = ''
        for key in params_dict.keys():
            out = out + key + ': ' + str(params_dict[key].item())
        out = out + f" loss: {loss.item()}"
        print(out)


mean: 5.114322662353516 loss: -0.7253590822219849
mean: 8.27955150604248 loss: -11.088034629821777
mean: 9.983960151672363 loss: -17.525123596191406
mean: 10.020857810974121 loss: -17.530719757080078
mean: 10.021407127380371 loss: -17.530717849731445
mean: 10.021413803100586 loss: -17.530723571777344
mean: 10.021413803100586 loss: -17.530723571777344
mean: 10.021413803100586 loss: -17.530723571777344
mean: 10.021413803100586 loss: -17.530723571777344
mean: 10.021413803100586 loss: -17.530723571777344


In [213]:
dist = Dist('x', GaussianMix([torch.tensor(1.)], [torch.tensor([5.])], [torch.tensor([[1.1920204162597656]])]))

neg_likelihood(dist, samples)

tensor([-27.6616])

In [183]:
print(output_dist)
neg_likelihood(output_dist, samples)

Dist<['a'],pi: [tensor(1.)]  mu: [tensor([5.])] sigma: [tensor([[0.8100]], grad_fn=<CopySlices>)]>


tensor([-29.6344], grad_fn=<NegBackward0>)